In [72]:
%pip install --quiet openai python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Deleting Contents in the JSON file

In [73]:
import json

def clear_sentiment_results(file_name):
    file_path = "results/" + file_name
    try:
        # Open the file in write mode, which will clear its contents
        with open(file_path, 'w') as f:
            # Write an empty list to the file
            json.dump([], f)
        print(f"All objects removed from {file_path}. The file now contains an empty list.")
    except Exception as e:
        print(f"An error occurred while clearing the file: {str(e)}")

# Call the function to clear the file
clear_sentiment_results("sentiment_results.json")

All objects removed from sentiment_results.json. The file now contains an empty list.


### New Code with tracking of reasonings and updating prompt

In [74]:
import json

def store_result(result, file_name):
    file_path = "results/" + file_name
    try:
        # Read existing results
        with open(file_path, 'r') as f:
            results = json.load(f)
    except FileNotFoundError:
        # If file doesn't exist, start with an empty list
        results = []
    
    # Append new result
    results.append(result)
    
    # Write updated results back to file
    with open(file_path, 'w') as f:
        json.dump(results, f, indent=2)

In [75]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()

# Access the API key
api_key = os.getenv('OPENAI_API_KEY')

# Create an OpenAI client
client = OpenAI(api_key=api_key)

# Define the sentiment schema
sentiment_schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "sentiment_response",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "sentiment": {
                    "type": "boolean",
                    "description": "True if in favor of the proposition, false if against"
                },
                "reason": {
                    "type": "string",
                    "description": "Very brief core argument in sentiment analysis"
                }
            },
            "required": ["sentiment", "reason"],
            "additionalProperties": False
        }
    }
}

def analyze_opinion_reasoning(question, opinion, previous_results):
    try:
        # Create a string of previous results and their frequencies
        previous_results_str = ", ".join([f"{reason}" for reason, _ in previous_results.items()])
        
        # TODO: Find a more organized way to display the previous results in prompt
        system_message = f"""You are a helpful assistant analyzing opinions on the following question: '{question}'.
        Analyze any opinion given and provide a boolean sentiment (true if the opinion answering yes to the question, false if is answering no) and a very brief core argument on why yes or no in a few words or less.
        If and only if the opinion's core argument is very similar to a previous core argument, use that exact wording.
        {f"Previous core arguments: {previous_results_str}." if previous_results else ''}
        Opinion: 
        """
        
        response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18", 
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": opinion}
            ],
            response_format=sentiment_schema,
            temperature=0.8,
            max_tokens=150  # Adjust as needed
        )
        result = json.loads(response.choices[0].message.content)
        
        # Update the previous_results dictionary
        reason = result.get('reason')
        if reason:
            previous_results[reason] = previous_results.get(reason, 0) + 1
        
        return result, previous_results
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [76]:
# Main execution
if __name__ == "__main__":
    file_name = "sentiment_results.json"
    question = "Should phones be banned from schools?"
    # opinions = [
    #     "I think phones should be banned from school since they are distracting from students learning.",
    #     "Phones are necessary for emergency situations, so they shouldn't be banned.",
    #     "Banning phones would eliminate cheating during exams.",
    #     "Students need phones to stay connected with their parents.",
    #     "Phones can be educational tools when used properly in the classroom.",
    # ]

    opinions =[
        "Just as more kids began spending more time with their phones, we saw a massive spike in depression and mental illness.",
        "Phones prevent socialization between students during school. ",
        "Despite what rules may exist, most students are using their phones during school",
        "Phone usage reduces learning",
        "having a smartphone with you at all times is it gives you the ability to instantly communicate with someone else. Students are able to contact parents, guardians or the authorities without much hassle and vice versa.",
        "Smartphones are all about the apps and the amazing things they can do, these apps can be used in a number of creative ways to facilitate their classroom learning experience.",
        "With phones, students can access research, news and videos to enhance their learning.",
        "Smartphones can be utilized for digital harassment in and out of school.",
        "students could become highly distracted from the many sources of entertainment.",
        "Frequent usage of smartphones has been linked to negative effects on both physical and mental health. ",
    ]


    # Initialize the previous_results dictionary
    previous_results = {}

    for opinion in opinions:
        json_content, previous_results = analyze_opinion_reasoning(question, opinion, previous_results)
        store_result(result=json_content, file_name=file_name)

    print("Core Argument Frequencies:")
    for reason, count in previous_results.items():
        print(f"{reason}: {count}")

Core Argument Frequencies:
Phones contribute to mental health issues.: 3
Phones prevent socialization between students during school.: 2
Phones reduce focus on education.: 2
Phones provide communication safety for students.: 3
